# Mean Field Approximation

Ejemplo de MFA (Inferencia Variacional)

In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.integrate import quad, dblquad
from scipy.optimize import minimize
from scipy.special import gamma


In [3]:
mu = 0
sigma = 1
N= 50
D = np.random.normal(mu,sigma,N)   
tol = 1e-6

In [22]:
## Calculos Varios
def E_Q_mu(D=D):
    return np.mean(D)
def E_Q_mu2(EspSigma,D=D,N=N):
    return E_Q_mu(D)+EspSigma/N
def E_Q_sigmaInv(VarMU,D=D,N=N):
    return np.mean(D**2)-2*np.mean(D)*E_Q_mu(D)+VarMU
    
## Densidades

def P(mu,sigma,N=N,D=D):
    lamb = N**2
    alpha = N/2
    m = np.mean(D)
    beta = np.sum(D**2)/2
    const =(beta**(alpha))*np.sqrt(lamb)/(gamma(alpha)*np.sqrt(2*np.pi)) 
    dep = (sigma**(alpha-0.5))*(np.exp(-beta*sigma))*(np.exp(-0.5*lamb*sigma*((mu-m)**2)))
    return const*dep

def Gamma(theta,a,b):
    return (b**(a))*(theta**(a-0.5))*(np.exp(-b*theta))/(gamma(a)) 

def KLDivergencia(Q,P):
    Integrand = lambda a,b : (Q(a,b)*np.log(Q(a,b)/P(a,b))) if P(a,b)>0 and Q(a,b)>0 else 0
    KL = dblquad(Integrand, -.3,1, lambda x: -1, lambda x: 1)[0]
    return KL

In [28]:
def Iteracion(tol):
    EsperanzaMu = E_Q_mu(D)
    VarianzaMu = 0
    dist = 1
    counter = 0
    while dist>tol:
        ### Update Esperanza y Varianza
        EsperanzaSigmaInv = E_Q_sigmaInv(VarianzaMu,D=D,N=N)
        VarianzaMu = E_Q_mu2(EsperanzaSigmaInv,D=D,N=N)
        
       #### Parametros Gamma 
    
        an = N/2
        bn = 0.5*(N*VarianzaMu-2*(N*EsperanzaMu)*EsperanzaMu+np.sum(D**2))
        
        Q_sigma = lambda sigma : Gamma(sigma,an,bn) if sigma >=0 else 0
        
        
        ### Parametros Gaussiana
        muN = EsperanzaMu
        lambdaNInv = EsperanzaSigmaInv/N
    
        Q_mu = lambda mu: stats.norm.pdf(mu,loc=muN,scale=lambdaNInv)
        
        ### Recuperar distribucion Original
        
        Q = lambda mu,sigma : Q_mu(mu)*Q_sigma(sigma) if Q_mu(mu)>=0 else 0
        
        dist = KLDivergencia(Q,P)
        counter+=1
    return (Q,counter)

In [29]:
Q=Iteracion(tol)
Q

(<function __main__.Iteracion.<locals>.<lambda>>, 1)

In [30]:
KLDivergencia(Q[0],P)

-0.07868901300637254

0.0024308789475163189